# Chapter 1. The World's Smallest Django Project

---

# TOC

## 1. Hello Django

### 1) Creating the View

### 2) The URL Patterns

### 3) The Settings

### 4) Running the Example

## 2. Improvements

### 1) WSGI Application

### 2) Additional Configuration

### 3) Reusable Template

---

## 1. Hello Django

While the **startproject** command is a useful tool, it is not required in order to start a Django project. You are free to lay out your project however you like based on what you want to do. For larger projects, developers benefit from the code organization provided by the startproject command. However, **the convenience of this command shouldn’t stop you from understanding what it does and why it is helpful.**
In this chapter we’ll lay out an example of how to create a simple project using Django’s basic building blocks.

This lightweight “Hello World” project will create a simple Django application **using a single-file approach.**

In this chapter, we’ll start by using a single ***hello.py*** file. This file will contain all of the code needed to run our Django project.

### 1) Creating the View

model-template-view (MTV) framework
- The view portion typically
    - inspects the incoming HTTP request
    - queries the necessary data to send to the presentation layer.
    - constructs the necessary data to send to the presentation layer.

**let’s create a simple way to execute a “Hello World” response.**

In [ ]:
%%writefile hello.py
from django.http import HttpResponse

def index(request):
    return HttpResponse('Hello World')

- this would typically be in a views.py file inside one of your apps.
- there is no requirement for views to live inside of apps.
- There is also no requirement that views live in a file called views.py.
- This is purely a matter of convention, but not a requirement on which to base our project’s structure.

### 2) The URL Patterns

- Django associates views with their URL by pairing a regular expression to match the URL and any callable arguments to the view.
- The following is an example from hello.py of how we make this connection.

In [ ]:
%%writefile hello.py
from django.conf.urls import url
from django.http import HttpResponse

def index(request):
    return HttpResponse('Hello World')

urlpatterns = (
    url(r'^$', index),
)

- Now this file combines both a typical views.py file and the root urls.py file.
- Again, it is worth noting that there is **no requirement for the URL patterns to be included in a urls.py file.**

### 3) The Settings

- **Django settings** detail everything from database and cache connections to internation‐ alization features and static and uploaded resources.
- For many developers just getting started, **the settings in Django are a major point of confusion.**
- While **recent releases have worked to trim down the default settings’ file length**, it can still be overwhelming.

In [ ]:
%%writefile hello.py
from django.conf import settings

settings.configure(
    DEBUG=True,
    SECRET_KEY='thisisthesecretkey',
    ROOT_URLCONF=__name__,
    MIDDLEWARE_CLASSES=(
        'django.middleware.common.CommonMiddleware',
        'django.middleware.csrf.CsrfViewMiddleware',
        'django.middleware.clickjacking.XFrameOptionsMiddleware',
    ),
)

from django.conf.urls import url
from django.http import HttpResponse

def index(request):
    return HttpResponse('Hello World')

urlpatterns = (
    url(r'^$', index),
)

- A **secret key** must be generated for the default session and cross-site request forgery (CSRF) protection.
- It is important for any production site to have a **random SECRET_KEY** that is kept private.
- https://docs.djangoproject.com/en/1.7/topics/signing

#### Cryptographic signing

The golden rule of Web application security is to never trust data from untrusted sources. **Cryptographically signed values** can be passed through an untrusted channel safe in the knowledge that any tampering will be detected.

**Django provides both a low-level API for signing values and a high-level API** for setting and reading signed cookies, one of the most common uses of signing in Web applications.

##### **Protecting the SECRET_KEY** ->  Securing signed data

When you create a new Django project using startproject, the **settings.py** file is generated automatically and gets a random **SECRET_KEY value**.

**This value is the key to securing signed data** – it is vital you keep this secure, or attackers could use it to generate their own signed values.

##### **Using the low-level API**

###### sign

In [ ]:
from django.core.signing import Signer
from django.core import signing

In [ ]:
signer = Signer()
value = signer.sign('My String')
print(value)

###### unsign

In [ ]:
original = signer.unsign(value)
print(original)

###### tampering

In [ ]:
value += 'm'
try:
    original = signer.unsign(value)
except signing.BadSignature:
    print("Tampering detected!!")

### 4) Running the Example

In [ ]:
%%writefile hello.py
import sys

# settings
from django.conf import settings

settings.configure(
    DEBUG=True,
    SECRET_KEY='thisisthesecretkey',
    ROOT_URLCONF=__name__,
    MIDDLEWARE_CLASSES=(
        'django.middleware.common.CommonMiddleware',
        'django.middleware.csrf.CsrfViewMiddleware',
        'django.middleware.clickjacking.XFrameOptionsMiddleware',
    ),
)

# views
from django.http import HttpResponse

def index(request):
    return HttpResponse('Hello World')

# urls
from django.conf.urls import url

urlpatterns = (
    url(r'^$', index),
)

if __name__ == "__main__":
    from django.core.management import execute_from_command_line
    execute_from_command_line(sys.argv)

---

## 2. Improvements

This example shows some of the fundamental pieces of the Django framework:
- writing views
- creating settings
- running management commands

**for taking incoming HTTP requests and returning HTTP responses.**

Django also provides additional utilities for common tasks such as:
- rendering HTML
- parsing form data
- persisting session state

### 1) WSGI Application

#### runserver

It has helpful utilities for **local development** such as auto–code reloading.

While it is convenient for local development, runserver is **not appropriate for production deployment** security.

#### Web Server Gateway Interface(WSGI)

is **the specification for how web servers communicate with application frameworks such as Django**, and was defined by PEP 333 and improved in PEP 3333.

- mod_wsgi
- Gunicorn
- uWSGI
- CherryPy
- Tornado
- Chaussette

Each of these servers needs a **properly defined WSGI application** to be used. Django has an easy interface for creating this application through **get_wsgi_application**.

In [ ]:
%%writefile hello.py
import sys

# settings
from django.conf import settings

settings.configure(
    DEBUG=True,
    SECRET_KEY='thisisthesecretkey',
    ROOT_URLCONF=__name__,
    MIDDLEWARE_CLASSES=(
        'django.middleware.common.CommonMiddleware',
        'django.middleware.csrf.CsrfViewMiddleware',
        'django.middleware.clickjacking.XFrameOptionsMiddleware',
    ),
)

# views
from django.http import HttpResponse

def index(request):
    return HttpResponse('Hello World')

# urls
from django.conf.urls import url

urlpatterns = (
    url(r'^$', index),
)

# wsgi
from django.core.wsgi import get_wsgi_application
application = get_wsgi_application()

if __name__ == "__main__":
    from django.core.management import execute_from_command_line
    execute_from_command_line(sys.argv)

This would normally be contained within the **wsgi.py** file created by the startproject command.

##### Gunicorn

**Gunicorn** is a **popular choice** for a pure-Python WSGI application server. (2015.09.02 버전 19.3.0)

```bash
$ pip install gunicorn
```

You can **run** it fairly simply

```bash
$ gunicorn hello --log-file=-
```

[docs.gunicorn.org](http://docs.gunicorn.org)

### 2) Additional Configuration

**While Gunicorn is a production-ready web server, the application itself is not yet production ready**, as **DEBUG** should never be enabled in production.

In [ ]:
%%writefile hello.py
import os
import sys

# settings
from django.conf import settings
DEBUG = os.environ.get('DEBUG', 'on') == 'on'
SECRET_KEY = os.environ.get('SECRET_KEY', os.urandom(32))

settings.configure(
    DEBUG=DEBUG,
    SECRET_KEY=SECRET_KEY,
    ROOT_URLCONF=__name__,
    MIDDLEWARE_CLASSES=(
        'django.middleware.common.CommonMiddleware',
        'django.middleware.csrf.CsrfViewMiddleware',
        'django.middleware.clickjacking.XFrameOptionsMiddleware',
    ),
)

# views
from django.http import HttpResponse

def index(request):
    return HttpResponse('Hello World')

# urls
from django.conf.urls import url

urlpatterns = (
    url(r'^$', index),
)

# wsgi
from django.core.wsgi import get_wsgi_application
application = get_wsgi_application()

if __name__ == "__main__":
    from django.core.management import execute_from_command_line
    execute_from_command_line(sys.argv)

>[Official doc.: os module](https://docs.python.org/3/library/os.html)

>- **os.environ**: A mapping object representing the string environment. For example, environ['HOME'] is the pathname of your home directory (on some platforms), and is equivalent to getenv("HOME") in C.

>- **os.urandom(n)**: Return a string of n random bytes suitable for cryptographic use.

The default for **DEBUG** is True, and the **SECRET_KEY** will be randomly generated each time the application is loaded if it is not set.

```bash
$ export DEBUG=off
$ python hello.py runserver
```

```bash
CommandError: You must set settings.ALLOWED_HOSTS if DEBUG is False.
```

In [3]:
%%writefile hello.py
import os
import sys

# settings
from django.conf import settings
DEBUG = os.environ.get('DEBUG', 'on') == 'on'
SECRET_KEY = os.environ.get('SECRET_KEY', os.urandom(32))
ALLOWED_HOSTS = os.environ.get('ALLOWED_HOSTS', 'localhost').split(',')

settings.configure(
    DEBUG=DEBUG,
    SECRET_KEY=SECRET_KEY,
    ALLOWED_HOSTS=ALLOWED_HOSTS,
    ROOT_URLCONF=__name__,
    MIDDLEWARE_CLASSES=(
        'django.middleware.common.CommonMiddleware',
        'django.middleware.csrf.CsrfViewMiddleware',
        'django.middleware.clickjacking.XFrameOptionsMiddleware',
    ),
)

# views
from django.http import HttpResponse

def index(request):
    return HttpResponse('Hello World')

# urls
from django.conf.urls import url

urlpatterns = (
    url(r'^$', index),
)

# wsgi
from django.core.wsgi import get_wsgi_application
application = get_wsgi_application()

if __name__ == "__main__":
    from django.core.management import execute_from_command_line
    execute_from_command_line(sys.argv)

Overwriting hello.py


### 3) Reusable Template

So far this example has centered on **rethinking the layout created by Django’s startproject command**.

**A template for startproject is a directory or zip file** that is rendered as a Django template when the command is run.

To transform hello.py into **a project template (project_name/ project_name.py)**, the relevant parts of the file need to be replaced by these variables.

In [10]:
!mkdir -p project_name

In [11]:
%%writefile project_name/project_name.py
import os
import sys

# settings
from django.conf import settings
DEBUG = os.environ.get('DEBUG', 'on') == 'on'
SECRET_KEY = os.environ.get('SECRET_KEY', ''{{ secret_key }}'')
ALLOWED_HOSTS = os.environ.get('ALLOWED_HOSTS', 'localhost').split(',')

settings.configure(
    DEBUG=DEBUG,
    SECRET_KEY=SECRET_KEY,
    ALLOWED_HOSTS=ALLOWED_HOSTS,
    ROOT_URLCONF=__name__,
    MIDDLEWARE_CLASSES=(
        'django.middleware.common.CommonMiddleware',
        'django.middleware.csrf.CsrfViewMiddleware',
        'django.middleware.clickjacking.XFrameOptionsMiddleware',
    ),
)

# views
from django.http import HttpResponse

def index(request):
    return HttpResponse('Hello World')

# urls
from django.conf.urls import url

urlpatterns = (
    url(r'^$', index),
)

# wsgi
from django.core.wsgi import get_wsgi_application
application = get_wsgi_application()

if __name__ == "__main__":
    from django.core.management import execute_from_command_line
    execute_from_command_line(sys.argv)

Writing project_name/project_name.py


Rather than using os.urandom for the SECRET_KEY default, **this code will generate a random secret to be the default each time a new project is created.** This makes the SECRET_KEY default stable at the project level while still being sufficiently random across projects.

In [12]:
!django-admin.py startproject foo --template=project_name

In [14]:
!ls -l

total 56
-rw-r--r--  1 initialkommit  staff  22872  9  3 11:30 Chapter_1_The_World's_Smallest_Django_Project.ipynb
drwxr-xr-x  3 initialkommit  staff    102  9  3 10:00 __pycache__
drwxr-xr-x  3 initialkommit  staff    102  9  3 11:31 foo
-rw-r--r--  1 initialkommit  staff    995  9  3 11:23 hello.py
drwxr-xr-x  3 initialkommit  staff    102  9  3 11:24 project_name
